# Check for test failures - Defects4j 2.0
This script is the second main script to be executed and it compares the gzoltar coverage results (that were be obtained via the auxiliary scripts) and the defects4j bug-triggering tests
Obs: In order to be able to execute this script, you must unzip the zip files that can be found in the locations data/gzoltar_files/ and data/gzoltar_files/Closure/

### Vars declaration

In [2]:
import os
from secrets import base_path


paths_dict=  {
        "bugs_details_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
        "gzoltar_files_path":  os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
        "output_file_path":  os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "rq1_results.json"),
}

### Getting the failing tests informations for each bug

In [7]:
import utils
import importlib
importlib.reload(utils)

bugs_data = utils.json_file_to_dict(paths_dict["bugs_details_file_path"])
bugs_with_failing_tests_in_commons_with_defects4j = {}
bugs_without_failing_tests_in_commons_with_defects4j = {}
bugs_with_coverage_problem = {}
bugs_with_no_coverage = []
for project in bugs_data.keys():
        print("----- " + project + " -----")
        project_gzoltar_folder = os.path.join(paths_dict["gzoltar_files_path"], project)
        for bug_id in bugs_data[project].keys():
                bug_report_id = project+ "_" + bug_id
                bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
                if not os.path.exists(bug_gzoltar_folder):
                        bugs_with_no_coverage.append(bug_report_id)
                        continue
                tests = utils.read_tests_file(bug_gzoltar_folder)
                number_not_passing_tests = len(tests["failing_tests"]) + len(tests["compilation_problems"])
                percentage_not_passing_tests = number_not_passing_tests / (number_not_passing_tests + len(tests["passing_tests"]))
                if len(tests["passing_tests"]) == 0 or percentage_not_passing_tests>0.1:
                        bugs_with_coverage_problem[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_falining_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"]
                        }
                        continue
                print(bug_report_id + ": "+ str(len(tests["failing_tests"])))
                found = False
                bugs_in_common_defects4j = []
                for failing_test in tests["failing_tests"]:
                        for defects4j_test in bugs_data[project][bug_id]["defects4j_trigger_tests"]:
                                defects4j_test = defects4j_test.replace("::", "#")
                                if failing_test == defects4j_test:
                                        found = True
                                        bugs_in_common_defects4j.append(failing_test)
                if found:
                        bugs_with_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_falining_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"],
                                "failing_tests_in_common_with_defects4j": bugs_in_common_defects4j
                        }
                else:
                        bugs_without_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_falining_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"]
                        }

output_dict = {
        "bugs_with_stack_traces": {
                "bugs_with_no_coverage": bugs_with_no_coverage,
                "bugs_with_coverage_problem": bugs_with_coverage_problem,
                "bugs_with_failing_tests_in_commons_with_defects4j": bugs_with_failing_tests_in_commons_with_defects4j,
                "bugs_without_failing_tests_in_commons_with_defects4j": bugs_without_failing_tests_in_commons_with_defects4j
        }
}
print("___________")
print("bugs_with_no_coverage: " + str(len(bugs_with_no_coverage)))
print("bugs_with_coverage_problem: " + str(len(bugs_with_coverage_problem)))
print(str(len(bugs_with_failing_tests_in_commons_with_defects4j)) + " out of " + str(len(bugs_without_failing_tests_in_commons_with_defects4j) + len(bugs_with_failing_tests_in_commons_with_defects4j)) + " bugs have tests in common with defects4j")

utils.dict_to_json_file(paths_dict["output_file_path"], output_dict)

----- Cli -----
Cli_14: 1
Cli_5: 3
Cli_7: 2
----- Closure -----
Closure_106: 1
Closure_125: 4
Closure_143: 1
Closure_152: 2
Closure_2: 3
Closure_34: 2
Closure_37: 2
Closure_55: 2
----- Codec -----
Codec_8: 0
----- Collections -----
Collections_28: 0
----- Compress -----
Compress_1: 0
Compress_12: 0
Compress_14: 0
Compress_17: 0
Compress_18: 0
Compress_23: 1
Compress_24: 3
Compress_27: 2
Compress_31: 2
Compress_32: 1
Compress_34: 2
----- Csv -----
Csv_12: 0
Csv_4: 0
----- Gson -----
Gson_12: 0
Gson_5: 0
Gson_8: 0
----- JacksonCore -----
JacksonCore_10: 1
JacksonCore_11: 0
JacksonCore_4: 1
JacksonCore_7: 0
----- JacksonDatabind -----
JacksonDatabind_14: 26
JacksonDatabind_15: 25
JacksonDatabind_25: 32
JacksonDatabind_28: 35
JacksonDatabind_3: 27
JacksonDatabind_32: 33
JacksonDatabind_37: 32
JacksonDatabind_59: 0
JacksonDatabind_73: 35
JacksonDatabind_9: 26
----- Jsoup -----
Jsoup_27: 0
Jsoup_34: 0
Jsoup_54: 0
Jsoup_78: 0
Jsoup_80: 0
Jsoup_82: 0
Jsoup_84: 1
Jsoup_90: 0
----- JxPath -----
